In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

from glob import glob
from natsort import natsorted
import pyvista as pv

from matplotlib.colors import Normalize
import matplotlib

import os

from glob import glob

from matplotlib import rc


# Set the font used for MathJax
rc('mathtext',**{'default':'regular'})
rc('figure',**{'figsize':(8,6)})

import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.family'] = 'Arial'

# rc('text',**{'usetex':True})

# plt.rcParams['text.usetex'] = True # TeX rendering

In [ ]:
current_ratio_U238_to_U235 = 137.818

half_life_U235 = 703.8e6  # Half-life of U-235 in years
half_life_U238 = 4.468e9  # Half-life of U-238 in years


current_U238_fraction = 0.992745
current_U235_fraction = 0.00720
current_ratio_U238_to_U235 = 137.818 ### from isoplotR


# Decay constant of U238
lambda_U238 = np.log(2) / half_life_U238
lambda_U235 = np.log(2) / half_life_U235

In [ ]:
from matplotlib import pyplot as plt, patches
import matplotlib.path as mpltPath

def concordia_curve(t, lambda_235=9.8485e-10, lambda_238=1.55125e-10):
    # Calculate the exponential growth for both decay systems
    e_lambda_235t = np.exp(lambda_235 * t)
    e_lambda_238t = np.exp(lambda_238 * t)
    
    # Calculate ratios
    Pb206_U238 = e_lambda_238t - 1
    Pb207_U235 = e_lambda_235t - 1
    Pb207_Pb206 = (Pb207_U235 / Pb206_U238) * (1 / current_ratio_U238_to_U235)  # Natural ratio of U235/U238

    return Pb207_Pb206, Pb206_U238
    
def create_sample_spot(centre, radius):

    ### create a circle at centre of box to represent garnet
    circle = patches.Circle(centre, radius)
    
    verts = circle.get_path().vertices
    trans = circle.get_patch_transform()
    circlePoints = trans.transform(verts)
    circleShape = mpltPath.Path(circlePoints)

    return circleShape

def sample_spot(spot, isotope, mesh):
    # with mesh.access(isotope):
    #     data = isotope.data[spot.contains_points(isotope.coords[:])] 
    # data = mesh[isotope][spot.contains_points(mesh.points[:])]
    data = mesh[isotope][spot.contains_points(mesh.points[:,0:2])]
    #     data = uw.utilities.gather_data(data, bcast=True)

    spot_average_data = np.average(data)


    return spot_average_data

def plot_zircon(directory, centre_points, sample_r, ax0, ax1, vmin0, vmax0, vmin1, vmax1):

    mesh = pv.read(natsorted(glob(f'{directory}*step*xdmf'))[-1])

    mesh_points = mesh.points[:,0:2]*100
    
    x = mesh.points[:,0]*100
    y = mesh.points[:,1]*100

    ## swarm
    # try:
    #     U238_Pb206 = (mesh['U238_U238'][:,0] / mesh['Pb206_Pb206'][:,0])
    #     Pb207_Pb206 = (mesh['Pb207_Pb207'][:,0] / mesh['Pb206_Pb206'][:,0])
    # except:
    #     U238_Pb206 = (mesh['U238_swarm'][:,] / mesh['Pb206_swarm'][:,])
    #     Pb207_Pb206 = (mesh['Pb207_swarm'][:,] / mesh['Pb206_swarm'][:,])
    # else:
    ### mesh
    U238_Pb206 = (mesh['U238_U238'] / mesh['Pb206_Pb206'])
    Pb207_Pb206 = (mesh['Pb207_Pb207'] / mesh['Pb206_Pb206'])
    
    U238_Pb206[np.isnan(U238_Pb206)] = 0
    U238_Pb206[U238_Pb206<0.05] = 0
    
    
    Pb207_Pb206[np.isnan(Pb207_Pb206)] = 0
    Pb207_Pb206[Pb207_Pb206<0.05] = 0
    
    
    
    # Create the Delaunay triangulation
    triang = tri.Triangulation(x, y)
    
    # Create the plot
    # fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)
    
    # cbar1 = ax0.tricontourf(triang, U238_Pb206, cmap='viridis', levels=np.linspace(vmin0, vmax0, 1000))
    cbar1 = ax0.scatter(x, y, c=U238_Pb206,cmap='viridis', s=10, vmin=vmin0, vmax=vmax0) #, vmin=10, vmax=25
    # Normalization object that clips values outside the range
    norm = Normalize(vmin=vmin0, vmax=vmax0, clip=True)
    plt.colorbar(cbar1,ax=ax0, extend='both', norm=norm )
    # cbar0.set_clim(10,25)

    
    
    # cbar1 = ax1.tricontourf(triang, Pb207_Pb206, cmap='viridis', levels=np.linspace(vmin1, vmax1, 1000))
    # cbar1.set_clim(0.052,0.057)
    cbar1 = ax1.scatter(x, y, c=Pb207_Pb206,cmap='viridis', s=10, vmin=vmin1, vmax=vmax1) #, vmin=0.052, vmax=0.05)
    plt.colorbar(cbar1,ax=ax1, extend='both')
    # cbar1.set_clim(0.052, 0.057)


    
    for axe in [ax0, ax1]:
        axe.set_aspect('equal', adjustable='box')
    
        axe.set_aspect('equal', adjustable='box')
        
        for centre in centre_points:
            spot = create_sample_spot(centre*100, sample_r*100)
    
            patch = patches.PathPatch(spot, ec=('k', 0.5), fc=(1,0,0,0), lw=1.5, ls='--')
            axe.add_patch(patch)
    
    
        axe.set_xlabel('x [$\mu m$] ')
    
    ax0.set_ylabel('y [$\mu m$] ')
    
    ax0.set_title(r'$\frac{^{238}U}{^{206}Pb}$')
    
    ax1.set_title(r'$\frac{^{207}Pb}{^{206}Pb}$')
    
    plt.tight_layout()
    
    # plt.savefig(f'{folder}/UPb-PbPb_plot2.pdf')


    return ax0, ax1


def extract_spot_data(fileName, centre_points, sample_r):
    ''' 
    order of array
    Pb206_Pb206, Pb207_Pb207, U235_U235, U238_U238
    '''
    mesh = pv.read(natsorted(glob(f'{fileName}'))[-1])

    try:
        mesh.point_data.remove('time')
    except:
        pass

    try:
        mesh.point_data.remove('zircon_regions_zircon_regions')
    except:
        pass
    
    spot_data = np.zeros(shape=(centre_points.shape[0], 4))
    
    i = 0
    for centre in centre_points:
        spot = create_sample_spot(centre, sample_r)
        x = 0
        for name in mesh.array_names:
            data = sample_spot(spot, name, mesh)
            spot_data[i, x] = data
            x+=1
    
        i += 1

    return spot_data

def extract_point_data(directory, points):
    mesh_data = pv.read(natsorted(glob(f'{directory}*step*xdmf'))[-1])

    spot_data = pv.PolyData(points).sample(mesh_data)

    return spot_data

In [ ]:
sample_r = 0.23 / 2 ### 23 micron diameter


sample_centre_points = np.array([[0.0, 0.0], [0.0, 0.3], [0.0, -0.37]])

sample_profile_points = np.array([[0.0, 0.0, 0.0], [0.125, 0.0, 0.0], [0.25, 0.0, 0.0]])

### Run isothermal & isotropic diffusion benchmark

### Plots and profiles from figure 5 are generated in the output file

options that can be set:
- uw_csize for cell size
- uw_degree for unknown degree
- uw_duration for model duration in Myrs
- uw_temp for temperature in C

In [ ]:
for temp in [750, 800, 850]:
    os.system(f'python3 Ex_isotropic_isothermalDiffusion-decay_zircon.py -uw_temp {temp}')

##### Extract data from each model

In [ ]:
isothermal_750_dir = f'./output/isotropic_isothermal_temp=750.0_duration=500.0_csize=0.01_Diffusion-Decay_test/'

isothermal_750_spot_data = extract_spot_data(f'{isothermal_750_dir}/*step*xdmf', sample_centre_points, sample_r)

In [ ]:
isothermal_800_dir = f'./output/isotropic_isothermal_temp=800.0_duration=500.0_csize=0.01_Diffusion-Decay_test/'

isothermal_800_spot_data = extract_spot_data(f'{isothermal_800_dir}/*step*xdmf', sample_centre_points, sample_r)

In [ ]:
isothermal_850_dir = f'./output/isotropic_isothermal_temp=850.0_duration=500.0_csize=0.01_Diffusion-Decay_test/'

isothermal_850_spot_data = extract_spot_data(f'{isothermal_850_dir}/*step*xdmf', sample_centre_points, sample_r)

In [ ]:
#### Create concordia plot (fig 5d)

In [ ]:
# Time range in years
time = np.linspace(100e6, 700e6, 1000)  # 0 to 1000 Ma

time1 = np.linspace(500e6, 600e6, 1)

# time2 = np.linspace(550e6, 1050e6, 6)



# Generate data for the Concordia curve
pb207_pb206, pb206_u238 = concordia_curve(time)

pb207_pb206_1, pb206_u238_1 = concordia_curve(time1)


# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='time marker', s=200)
plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
for i in range(len(pb206_u238_1)):
    plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')


plt.scatter(isothermal_750_spot_data[:,3]/isothermal_750_spot_data[:,0], isothermal_750_spot_data[:,1]/isothermal_750_spot_data[:,0], marker='v', c='k', s = 150, label='750 $\degree$C')



plt.scatter(isothermal_800_spot_data[:,3]/isothermal_800_spot_data[:,0], isothermal_800_spot_data[:,1]/isothermal_800_spot_data[:,0], marker='^', c='green', s = 150, alpha=0.6, label='800 $\degree$C')

plt.scatter(isothermal_850_spot_data[:,3]/isothermal_850_spot_data[:,0], isothermal_850_spot_data[:,1]/isothermal_850_spot_data[:,0], marker='d', c='lightblue', s = 150, alpha=0.4, label='850 $\degree$C')

# plt.scatter(decreasingT_spot_data[:,3]/decreasingT_spot_data[:,0], decreasingT_spot_data[:,1]/decreasingT_spot_data[:,0], marker='o', c='red', s = 150, alpha=0.15, label='850 to 750 $\degree$C')



plt.legend()
plt.xlabel(r'$\frac{^{238}U}{^{206}Pb}$', labelpad=10)
plt.ylabel(r'$\frac{^{207}Pb}{^{206}Pb}$', rotation=0, labelpad=10)
# plt.title('U-Pb Concordia')
plt.xlim(11., 15)
plt.ylim(0.055, 0.06)
# plt.legend()
plt.grid(True)


# plt.savefig('./output/U-Pb_isotope_concordia_plot_spot_data.pdf')

plt.show()

##### Create zircon plot (fig 5e)

In [ ]:
# Create the plot


fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)

            
plot_zircon(isothermal_850_dir, sample_centre_points, sample_r, ax[0], ax[1], vmin0=10, vmax0=25, vmin1=0.052, vmax1=0.057)

# plt.savefig(f'{isothermal_850_dir}850C_zircon_results+sample_spots_tricontourf.pdf', bbox_inches='tight')

### Run partial and full lead loss models

- Code to replicate figure 6
- Temperature profiles are generated within the .py script

In [ ]:
for test_profile in [1, 2, 3, 4]:
    os.system(f'python3 Ex_isotropic_temp-profiles_Diffusion-decay_zircon.py -uw_test_profile {test_profile}')

In [ ]:
sample_r = 0.23 / 2 ### 23 micron diameter


x_lower, x_upper = -0.3+sample_r, 0.3-sample_r
y_lower, y_upper = -0.5+sample_r, 0.5-sample_r

### Set a seed for reproducibility

np.random.seed(0)
sample_centre_points = np.random.uniform(low=(x_lower, y_lower), high=(x_upper, y_upper), size=(120, 2))

In [ ]:
profiles_dir = natsorted(glob('./output/isotropic_temp-profile=*_duration=1500Myr_tPeak=1000Myr_csize=0.01_udeg=1_Diffusion-Decay_test/'))
profiles_dir

In [ ]:
profile1_spot_data = extract_spot_data(fileName=f'{profiles_dir[0]}/*step*xdmf', centre_points=sample_centre_points, sample_r=sample_r)

profile2_spot_data = extract_spot_data(fileName=f'{profiles_dir[1]}/*step*xdmf', centre_points=sample_centre_points, sample_r=sample_r)

profile3_spot_data = extract_spot_data(fileName=f'{profiles_dir[2]}/*step*xdmf', centre_points=sample_centre_points, sample_r=sample_r)

profile4_spot_data = extract_spot_data(fileName=f'{profiles_dir[3]}/*step*xdmf', centre_points=sample_centre_points, sample_r=sample_r)

In [ ]:
# Time range in years
time = np.linspace(100e6, 1700e6, 1000)  # 0 to 1000 Ma

time1 = np.linspace(500e6, 1500e6, 11)

# time2 = np.linspace(550e6, 1050e6, 6)



# Generate data for the Concordia curve
pb207_pb206, pb206_u238 = concordia_curve(time)

pb207_pb206_1, pb206_u238_1 = concordia_curve(time1)


# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='Time', s=200)
plt.plot(1/pb206_u238, pb207_pb206, label='Concordia Curve', c='k')
for i in range(len(pb206_u238_1)):
    plt.text( (1/pb206_u238_1[i])-0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')

plt.plot([ 1/pb206_u238_1[0], 1/pb206_u238_1[-1] ], [ pb207_pb206_1[0], pb207_pb206_1[-1] ], ls='--', c='k', label='Disconcordant line')









# plt.scatter(profile3_HR_spot_data[:,3]/profile3_HR_spot_data[:,0], profile3_HR_spot_data[:,1]/profile3_HR_spot_data[:,0], marker='o', c='blue', s = 150, alpha=0.6, label='Profile 2')



# plt.scatter(profile2_HR_spot_data[:,3]/profile2_HR_spot_data[:,0], profile2_HR_spot_data[:,1]/profile2_HR_spot_data[:,0], marker='o', c='blue', s = 150, alpha=0.6, label='Profile 2')


plt.scatter(profile1_spot_data[:,3]/profile1_spot_data[:,0], profile1_spot_data[:,1]/profile1_spot_data[:,0], marker='s', s = 150, alpha=0.1, label='Profile 1')

plt.scatter(profile2_spot_data[:,3]/profile2_spot_data[:,0], profile2_spot_data[:,1]/profile2_spot_data[:,0], marker='^', s = 150, alpha=0.6, label='Profile 2')

plt.scatter(profile3_spot_data[:,3]/profile3_spot_data[:,0], profile3_spot_data[:,1]/profile3_spot_data[:,0], marker='v', s = 150, alpha=0.4, label='Profile 3')

plt.scatter(profile4_spot_data[:,3]/profile4_spot_data[:,0], profile4_spot_data[:,1]/profile4_spot_data[:,0], marker='d', s = 150, alpha=0.4, label='Profile 4')


plt.legend()
plt.xlabel(r'$\frac{^{238}U}{^{206}Pb}$', labelpad=10)
plt.ylabel(r'$\frac{^{207}Pb}{^{206}Pb}$', rotation=0, labelpad=10)
# plt.title('U-Pb Concordia')
plt.xlim(3.5, 14)
plt.ylim(0.05, 0.1)
plt.grid(True)


# plt.savefig('../output/T_profiles-U-Pb_isotope_concordia_plot-spot_data.pdf')

plt.show()

### Run secondary growth model

In [ ]:
os.system(f'python3 Ex_isotropic_Diffusion-decay_zircon-dual-growth.py')

In [ ]:
dual_growth_dir = f'./output/isotropic_dual-growth_duration=1500_2nd_growth=1000Myr_csize=0.01_Diffusion-Decay_test/'

In [ ]:
outer_zircon_sample_locations = np.array([[-45, 30],
                                        [-45, 20],
                                        [-45, 10],
                                        [-45, 0],
                                        [-45, -10],
                                        [-45, -20],
                                        [-45, -30],
                                        [45, 30],
                                        [45, 20],
                                        [45, 10],
                                        [45, 0],
                                        [45, -10],
                                        [45, -20],
                                        [45, -30],
                                        [0, -65],
                                        [0,70],
                                        [-25, -60],
                                        [25, -60],
                                        [-25, 60],
                                        [25, 60]])


zircon_interface = [(-0.3, 0.35, 0), (-0.2, 0.5, 0), (0.2, 0.5, 0), (0.3, 0.35, 0), 
                      (0.3, -0.35, 0), (0.2, -0.5, 0), (-0.2, -0.5, 0), (-0.3,-0.35, 0), (-0.3, 0.35, 0)]

# Generate 20 points on the given line
points_on_line = []
for i in range(len(zircon_interface) - 1):
    start = zircon_interface[i]
    end = zircon_interface[i + 1]
    line_points = np.linspace(start, end, 5, endpoint=False)
    points_on_line.extend(line_points)

zircon_interface_sample_locations = np.array(points_on_line)[:,0:2]


# sample_profile_points = pv.PolyData(np.array([[0.0, 0.0, 0.0], [0.125, 0.0, 0.0], [0.25, 0.0, 0.0]]))

In [ ]:
dual_growth_inner_spot_data = extract_spot_data(f'{dual_growth_dir}/*step*xdmf', sample_centre_points, sample_r)


dual_growth_interface_spot_data = extract_spot_data(f'{dual_growth_dir}/*step*xdmf', zircon_interface_sample_locations, sample_r)


dual_growth_outer_spot_data = extract_spot_data(f'{dual_growth_dir}/*step*xdmf', outer_zircon_sample_locations/100, sample_r)

In [ ]:
# Time range in years
time = np.linspace(100e6, 1700e6, 1000)  # 0 to 1000 Ma

time1 = np.linspace(500e6, 1500e6, 11)

# time2 = np.linspace(550e6, 1050e6, 6)



# Generate data for the Concordia curve
pb207_pb206, pb206_u238 = concordia_curve(time)

pb207_pb206_1, pb206_u238_1 = concordia_curve(time1)


# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(1/pb206_u238_1, pb207_pb206_1, c='orange', label='Time', s=200)
plt.plot(1/pb206_u238, pb207_pb206, label='Concordia curve', c='k')
for i in range(len(pb206_u238_1)):
    plt.text( (1/pb206_u238_1[i])+0.1, pb207_pb206_1[i]+0.0002, f'{round(time1[i]/1e6)} Ma')


# plt.scatter(profile1_point_data['U238_U238']/profile1_point_data['Pb206_Pb206'], profile1_point_data['Pb207_Pb207']/profile1_point_data['Pb206_Pb206'], marker='v', c='k', s = 150, label='750 $\degree$C')

plt.plot([ 1/pb206_u238_1[0], 1/pb206_u238_1[-1] ], [ pb207_pb206_1[0], pb207_pb206_1[-1] ], ls='--', c='k', label='Disconcordant line')




plt.scatter(dual_growth_inner_spot_data[:,3]/dual_growth_inner_spot_data[:,0], dual_growth_inner_spot_data[:,1]/dual_growth_inner_spot_data[:,0], marker='v', c='green', s = 250, alpha=0.9, label='Inner')

plt.scatter(dual_growth_interface_spot_data[:,3]/dual_growth_interface_spot_data[:,0], dual_growth_interface_spot_data[:,1]/dual_growth_interface_spot_data[:,0], marker='d', c='lightblue', s = 150, alpha=0.4, label='Interface')


plt.scatter(dual_growth_outer_spot_data[:,3]/dual_growth_outer_spot_data[:,0], dual_growth_outer_spot_data[:,1]/dual_growth_outer_spot_data[:,0], marker='^', c='red', s = 150, alpha=0.4, label='Outer')
# plt.scatter(decreasingT_point_data['U238_U238']/decreasingT_point_data['Pb206_Pb206'], decreasingT_point_data['Pb207_Pb207']/decreasingT_point_data['Pb206_Pb206'], marker='o', c='red', s = 150, alpha=0.15, label='850 to 750 $\degree$C')


# plt.scatter(decreasingT_point_data['U238_U238']/decreasingT_point_data['Pb206_Pb206'], decreasingT_point_data['Pb207_Pb207']/decreasingT_point_data['Pb206_Pb206'], marker='o', c='red', s = 150, alpha=0.15, label='850 to 750 $\degree$C')


plt.scatter(profile4_spot_data[:,3]/profile4_spot_data[:,0], profile4_spot_data[:,1]/profile4_spot_data[:,0], marker='o', c='blue', s = 150, alpha=0.2, label='single growth (profile 4)')


plt.legend()
plt.xlabel(r'$\frac{^{238}U}{^{206}Pb}$', labelpad=10)
plt.ylabel(r'$\frac{^{207}Pb}{^{206}Pb}$', rotation=0, labelpad=10)
# plt.title('U-Pb Concordia')
plt.xlim(3.5, 15)
plt.ylim(0.05, 0.1)
plt.grid(True)


# plt.savefig(f'{dual_growth_dir}dualGrowth_zircon-U-Pb_isotope_concordia_plot.pdf', bbox_inches='tight')

plt.show()